Aero Rims
===
Simulate hitting an object riding a bike. 

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo
from ngsolve.krylovspace import CGSolver
from netgen.occ import *
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

importing NGSolve-6.2.2301


In [2]:
dim = 3
h = 10
geo = OCCGeometry('aero-rims-w-axle.stp', dim=dim)
mesh = Mesh(geo.GenerateMesh(maxh=h)).Curve(1)
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [3]:
print("boundaries: ", mesh.GetBoundaries())
print("materials: ", mesh.GetMaterials())
print("number elements: ", mesh.ne)

boundaries:  ('Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörper', 'Hauptkörpe

In [4]:
order = 1
fes_p = L2(mesh, order=order+1, all_dofs_together=True, dgjumps=True) 
fes_u = VectorL2(mesh, order=order, piola=True, dgjumps=True)
fes_tr = FacetFESpace(mesh, order=order+1)
fes = fes_p * fes_u

traceop = fes_p.TraceOperator(fes_tr, average=True) 

gfu = GridFunction(fes_u)
gfp = GridFunction(fes_p)
gftr = GridFunction(fes_tr)

gfp.Set( exp(-1/400*((x-325*cos(45))**2+(y-0)**2+(z-325*sin(45))**2)))
gftr.vec.data = traceop * gfp.vec

p, u = fes.TrialFunction()
q, v = fes.TestFunction()
phat = fes_tr.TrialFunction()

n = specialcf.normal(mesh.dim)
dS = dx(skeleton=True) 
def jump(p): return p.Other()-p
def avgn(v): return 0.5*(v*n-v.Other()*n.Other())

In [5]:
embp, embu = fes.embeddings

In [6]:
tend = 5
dt = 1e-1
print ("dt = ", dt)
print("Number of steps: ", tend/dt)

dt =  0.1
Number of steps:  50.0


In [7]:
print("fes ndofs: ", fes.ndof)
print("fes_u ndofs: ", fes_u.ndof)
print("fes_p ndofs: ", fes_p.ndof)
print("fes_tr ndofs: ", fes_tr.ndof)

fes ndofs:  3322616
fes_u ndofs:  1812336
fes_p ndofs:  1510280
fes_tr ndofs:  2064768


In [8]:
if dim == 2:
    Draw (gfp, order=3)
else:
    gftr.vec.data = traceop * gfp.vec
    Draw (gftr, draw_vol=False, order=3);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [9]:
mesh.ne

151028

In [10]:
F = specialcf.JacobianMatrix(mesh.dim)
Finv = Inv(F)
detF = Det(F)
Norm_Finv = Norm(Finv)
el_norms = Integrate(Norm_Finv*1/detF.Norm(), mesh, element_wise=True)
el_norms_numpy = np.array(el_norms)

sorted_el_norms = -np.sort(-el_norms_numpy) #sort descending
# ref_index = int(4*sqrt(len(sorted_el_norms))) # sqrt(n) elements as implcit where n is mesh.ne,
# ref_norm = sorted_el_norms[ref_index]
ref_norm = el_norms_numpy.mean()*1.4# using elements > mean 
impl_els = np.where(el_norms_numpy > ref_norm, 1, 0)
    
print('max', max(el_norms_numpy))
print('min', min(el_norms_numpy))
print('mean', el_norms_numpy.mean())
print('median', np.median(el_norms_numpy))
print('ref_norm', ref_norm)

max 7.654216259927644
min 0.02090160820817572
mean 0.29035692621612563
median 0.28841966713789413
ref_norm 0.40649969670257585


In [11]:
ba_implicit_els = BitArray(mesh.ne)
ba_explicit_els = BitArray(mesh.ne)
ba_interface_edges = BitArray(mesh.nedge)
ba_explicit_edges = BitArray(mesh.nedge)
ba_implicit_edges = BitArray(mesh.nedge)

ba_implicit_els[:] = 0
ba_explicit_els[:] = 0
ba_interface_edges[:] = 0
ba_explicit_edges[:] = 0
ba_implicit_edges[:] = 0

for el in mesh.Elements():
    if impl_els[el.nr] == 1:
        ba_implicit_els[el.nr] = 1
        for e in el.edges:
            ba_implicit_edges[e.nr] = 1
    else:
        ba_explicit_els[el.nr] = 1
        for e in el.edges:
            ba_explicit_edges[e.nr] = 1
        
ba_interface_edges = ba_explicit_edges & ba_implicit_edges     

In [12]:
ba_local_implicit_dofs = BitArray(fes.ndof)
ba_local_implicit_dofs[:] = 0

for el in mesh.Elements():
    if ba_implicit_els[el.nr] == 1:
        for nr in fes.GetDofNrs(el):
            ba_local_implicit_dofs[nr] = 1
    if ba_explicit_els[el.nr] == 1:
        for e in el.edges:
            if ba_interface_edges[e.nr] == 1:
                for nr in fes.GetDofNrs(el):
                    ba_local_implicit_dofs[nr] = 1

ba_local_implicit_dofs_u = BitArray(fes_u.ndof)
ba_local_implicit_dofs_u[:] = 0

for el in mesh.Elements():
    if ba_implicit_els[el.nr] == 1:
        for nr in fes_u.GetDofNrs(el):
            ba_local_implicit_dofs_u[nr] = 1
    if ba_explicit_els[el.nr] == 1:
        for e in el.edges:
            if ba_interface_edges[e.nr] == 1:
                for nr in fes_u.GetDofNrs(el):
                    ba_local_implicit_dofs_u[nr] = 1

Combine linear operators:

In [13]:
A = BilinearForm(fes)
A += -p*q*dx +u*v*dx 
A += dt/2*(grad(p)*v + grad(q)*u)*dx
A += dt/2*(jump(p)*avgn(v)+jump(q)*avgn(u)) * dS
A.Assemble()

p1, q1 = fes_p.TnT()
u1, v1 = fes_u.TnT()
phat1, qhat1 = fes_tr.TnT()
Bel = BilinearForm(trialspace=fes_p, testspace=fes_u, geom_free = True)
Bel += grad(p1)*v1 * dx -p1*(v1*n) * dx(element_boundary=True)
Bel.Assemble()
Btr = BilinearForm(trialspace=fes_tr, testspace=fes_u, geom_free = True)
Btr += phat1 * (v1*n) *dx(element_boundary=True)
Btr.Assemble();

B = Bel.mat + Btr.mat @ traceop
B_T = B.T

In [14]:
massu = fes_u.Mass(1)
invmassu = fes_u.Mass(1).Inverse()
invmassp = fes_p.Mass(1).Inverse()

In [15]:
Ps = Projector(ba_local_implicit_dofs_u, True)   # projection to small
Pl = Projector(ba_local_implicit_dofs_u, False)  # projection to large
B_e = Pl @ B
B_i = Ps @ B
print ("shape B_e: ", B_e.shape)
print ("shape B_i: ", B_i.shape)
print ("shape Ps: ", Ps.shape)

shape B_e:  (558468, 465390)
shape B_i:  (558468, 465390)
shape Ps:  (558468, 558468)


In [16]:
print ("local implicit dofs: ", ba_local_implicit_dofs.NumSet(),"/",len(ba_local_implicit_dofs))
print ("local implicit dofs of fes u: ", ba_local_implicit_dofs_u.NumSet(),"/",len(ba_local_implicit_dofs_u))

local implicit dofs:  262680 / 1023858
local implicit dofs of fes u:  143280 / 558468


In [17]:
invA = A.mat.Inverse(freedofs=ba_local_implicit_dofs, inverse="sparsecholesky")

# delete non zeros elements of matrix to speed up matrix multiplication
Anze = A.mat.DeleteZeroElements(10e-12)
invAnze = Anze.Inverse(freedofs=ba_local_implicit_dofs, inverse="sparsecholesky")
invmstar = embu.T @ invAnze @ embu
mstarloc = massu + dt*dt/4*B_i @ invmassp @ B_T

In [18]:
A.mat.nze, invA.nze, Anze.nze

(101951212, 36451800, 20098142)

In [19]:
invmassuB = invmassu @ B
invmasspB_T = invmassp @ B_T
invmstar_mstar = invmstar @ mstarloc

In [20]:
# visualize implicit dofs
from time import sleep
ba_gfu = BitArray(fes.ndof)
ba_gfu[:] = 0
gfuim = GridFunction(fes)
gfuim.vec[:] = 0
for i in range(len(ba_local_implicit_dofs)):
    if ba_local_implicit_dofs[i] == 1:
        gfuim.vec.data[i] = 1

if dim ==2:
    Draw(gfuim.components[0], mesh)
else:
    gftrtest = GridFunction(fes_tr)
    gftrtest.vec.data = traceop * gfuim.components[0].vec
    Draw(gftrtest, draw_vol=False, order=3);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [23]:
if dim == 2:
    scene = Draw (gfp, order=3, deformation=True);
else:
    scene = Draw (gftr, draw_vol=False, order=3);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [24]:
gfp.Set( exp(-1/400*((x-325*cos(45))**2+(y-0)**2+(z-325*sin(45))**2)))
gfu.vec[:] = 0
gfp_halfstep = gfp.vec.CreateVector()
gfuold = gfu.vec.CreateVector()
res = gfu.vec.CreateVector()
 
tend = 500    

t = 0
cnt = 0
loop = tqdm([i*dt for i in range(int(tend/dt))])
with TaskManager():#pajetrace=10**8):
    for i in loop:
        #t = t+dt
        
        gfp_halfstep.data = gfp.vec - dt/2 * invmassp @ B.T * gfu.vec
        
        res.data = dt * B_e * gfp_halfstep + dt/2 * B_i * (gfp_halfstep + gfp.vec) + massu * gfu.vec
        gfuold.data = invmassu * res 
        gfu.vec.data = gfuold
        gfu.vec.data += invmstar * (res - mstarloc * gfuold)
        gfp.vec.data = gfp_halfstep - dt/2 * invmassp @ B.T * gfu.vec
        cnt = cnt+1
        if cnt%10 == 0:
            if dim == 3:
                gftr.vec.data = traceop * gfp.vec
            scene.Redraw()
        loop.set_description(f'Time step {i/dt:.0f}')
        loop.set_postfix_str(f'Time {i:.4f} seconds')

Time step 4999: 100%|███████████████████████████████████████| 5000/5000 [09:05<00:00,  9.17it/s, Time 499.9000 seconds]
